Analyzing COCO Yolo Model in Roboflow by detecting Cats Images with Segmentation Techniques

In [1]:
pip install requests matplotlib pillow inference-sdk


In [2]:
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from io import BytesIO
import json
from inference_sdk import InferenceHTTPClient

# Initialize the Roboflow Inference Client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="IJgd8jCLHIdJE3mNjMvJ"
)

def display_image_with_boxes(image_path, detections):
    """
    Display an image with bounding boxes overlaid.
    Args:
        image_path (str): Path to the image file.
        detections (list): List of detection results containing bounding box coordinates.
    """
    # Open the image
    img = Image.open(image_path)
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # Draw each bounding box
    for detection in detections:
        x, y, w, h = (
            detection["x"],
            detection["y"],
            detection["width"],
            detection["height"],
        )
        label = detection["class"]
        confidence = detection["confidence"]

        # Calculate bounding box coordinates
        left = x - (w / 2)
        top = y - (h / 2)

        # Create a rectangle patch
        rect = patches.Rectangle(
            (left, top), w, h, linewidth=2, edgecolor="red", facecolor="none"
        )
        ax.add_patch(rect)
        plt.text(left, top, f"{label}: {confidence:.2f}", color="yellow", fontsize=8)

    plt.axis("off")
    plt.show()

def main():
    # Path to the image to be detected
    image_path = "sample.jpg"  # Replace with the path to your image file

    # Run inference using Roboflow
    try:
        print("Running inference...")
        result = CLIENT.infer(image_path, model_id="coco-0qrql/1")

        # Parse detection results
        detections = result["predictions"]
        print("Detections:", json.dumps(detections, indent=2))

        # Display the image with bounding boxes
        display_image_with_boxes(image_path, detections)

    except Exception as e:
        print("Error during inference:", e)

if __name__ == "__main__":
    main()


Running inference...
Error during inference: HTTPCallErrorError(description='400 Client Error: Bad Request for url: https://detect.roboflow.com/coco-0qrql/1?api_key=IJ***vJ&disable_active_learning=False', api_message='Could not load input image. Cause: Malformed base64 input image.',status_code=400)


In [3]:
import cv2
import matplotlib.pyplot as plt

# Load the image in color (BGR format)
img = cv2.imread(r'/content/sample.jpg')

# Convert the BGR image to grayscale for Otsu's thresholding
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Otsu's thresholding
ret, thresh = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Display the original (color) and thresholded (grayscale) images using matplotlib
plt.figure(figsize=(10, 5))

# Original color image
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
plt.title('Original Color Image')
plt.axis('off')

# Otsu Thresholding result
plt.subplot(1, 2, 2)
plt.imshow(thresh, cmap='gray')
plt.title("Otsu's Thresholding (Grayscale)")
plt.axis('off')

# Show the images
plt.show()

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load the image in color
img = cv2.imread(r'/content/sample.jpg')

# Convert to grayscale for adaptive thresholding
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply mean adaptive thresholding
thresh1 = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

# Apply Gaussian adaptive thresholding
thresh2 = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Display the original and thresholded images using matplotlib
plt.figure(figsize=(15, 5))

# Original image (color)
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
plt.title('Original Image')
plt.axis('off')

# Mean Adaptive Thresholding result
plt.subplot(1, 3, 2)
plt.imshow(thresh1, cmap='gray')
plt.title('Mean Adaptive Thresholding')
plt.axis('off')

# Gaussian Adaptive Thresholding result
plt.subplot(1, 3, 3)
plt.imshow(thresh2, cmap='gray')
plt.title('Gaussian Adaptive Thresholding')
plt.axis('off')

# Show the images
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def region_growing(image, seed_point, threshold):
    """
    Performs region growing on an image.

    Args:
        image: The input image (grayscale).
        seed_point: The coordinates of the seed point.
        threshold: The similarity threshold.

    Returns:
        A binary mask representing the segmented region.
    """
    rows, cols = image.shape
    mask = np.zeros_like(image)
    queue = [seed_point]

    while queue:
        x, y = queue.pop(0)
        if mask[x, y] == 0:
            mask[x, y] = 1
            neighbors = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]
            for nx, ny in neighbors:
                if 0 <= nx < rows and 0 <= ny < cols and abs(int(image[x, y]) - int(image[nx, ny])) <= threshold:
                    queue.append((nx, ny))

    return mask

# Example usage
image = cv2.imread(r'/content/sample.jpg')  # Load in color
seed_point = (100, 100)  # Set the seed point
threshold = 10  # Set the similarity threshold

# Convert the image to grayscale for region growing
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
mask = region_growing(gray_image, seed_point, threshold)

# Display the original color image and region growing result using matplotlib
plt.figure(figsize=(10, 5))

# Original image (in color)
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
plt.title('Original Image')
plt.axis('off')

# Region Growing result
plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray')
plt.title('Region Growing')
plt.axis('off')

# Show the images
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Load the image in color (BGR format)
image = cv2.imread(r'/content/sample.jpg')

# Reshape the image to a 2D array of pixels (3 channels)
img_reshaped = image.reshape((-1, 3))

# Perform K-means clustering
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(img_reshaped)

# Get the labels and cluster centers
labels = kmeans.labels_
centers = kmeans.cluster_centers_.astype(int)

# Reshape the labels back to the original image shape
labels = labels.reshape(image.shape[:2])

# Create the segmented image using the cluster centers
segmented_image = np.zeros_like(image)
for i in range(len(centers)):
    segmented_image[labels == i] = centers[i]

# Display the original and segmented images using matplotlib
plt.figure(figsize=(10, 5))

# Original image
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
plt.title('Original Image')
plt.axis('off')

# Segmented image
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
plt.title('K-Means Clustering')
plt.axis('off')

# Show the images
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift

# Load the image
img = cv2.imread(r'/content/sample.jpg')

# Resize the image to a smaller size (for example, to 100x100)
img_resized = cv2.resize(img, (100, 100))

# Reshape the resized image to a 2D array (each pixel is a point)
img_reshaped = img_resized.reshape((-1, 3))

# Perform mean shift clustering
ms = MeanShift()
ms.fit(img_reshaped)
labels = ms.labels_
centers = ms.cluster_centers_

# Reshape the labels back to the 2D image shape
labels = labels.reshape(img_resized.shape[:2])

# Create the segmented image by assigning the cluster centers to each pixel
segmented_image = np.zeros_like(img_resized)
for i in range(len(centers)):
    segmented_image[labels == i] = centers[i]

# Display the original (resized) and segmented images using matplotlib
plt.figure(figsize=(10, 5))

# Resized original image
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
plt.title('Original Resized Image')
plt.axis('off')

# Mean Shift Clustering result
plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(segmented_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
plt.title("Mean Shift Clustering")
plt.axis('off')

# Show the images
plt.show()

OTSU - Cat Detect

In [ ]:
import cv2
import matplotlib.pyplot as plt
from inference_sdk import InferenceHTTPClient
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Initialize the Roboflow Client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="IJgd8jCLHIdJE3mNjMvJ"
)

def save_temp_image(image, path="temp_thresh_image.jpg"):
    """
    Save the thresholded image temporarily for detection.
    Args:
        image (numpy array): Otsu thresholded image.
        path (str): Path to save the image.
    """
    cv2.imwrite(path, image)
    return path

def run_inference(image_path):
    """Run object detection using Roboflow API and return detections."""
    result = CLIENT.infer(image_path, model_id="coco-0qrql/1")  # Replace model ID if needed
    return result["predictions"]

def display_image_with_boxes(image_path, detections):
    """Display the thresholded image with bounding boxes from detections."""
    # Open the image using PIL
    img = Image.open(image_path).convert("RGB")  # Ensure it's RGB for visualization
    fig, ax = plt.subplots(1, figsize=(10, 6))
    ax.imshow(img, cmap="gray")

    # Add bounding boxes
    for detection in detections:
        x, y, w, h = detection["x"], detection["y"], detection["width"], detection["height"]
        label = detection["class"]
        confidence = detection["confidence"]

        # Convert to bounding box (left, top)
        left = x - w / 2
        top = y - h / 2

        # Draw bounding box
        rect = patches.Rectangle((left, top), w, h, linewidth=2, edgecolor="red", facecolor="none")
        ax.add_patch(rect)
        plt.text(left, top - 5, f"{label}: {confidence:.2f}", color="yellow", fontsize=8)

    plt.axis('off')
    plt.title("Object Detection on Otsu Thresholded Image")
    plt.show()

def main():
    # Path to the original image
    image_path = r'/content/sample.jpg'

    # Step 1: Load the image
    img = cv2.imread(image_path)

    # Step 2: Convert the image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Step 3: Apply Otsu's thresholding
    ret, thresh = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Step 4: Save the Otsu thresholded image temporarily
    thresh_image_path = save_temp_image(thresh)

    # Step 5: Run object detection using Roboflow API
    print("Running object detection on Otsu thresholded image...")
    detections = run_inference(thresh_image_path)
    print("Detections:", detections)

    # Step 6: Display the Otsu thresholded image with bounding boxes
    display_image_with_boxes(thresh_image_path, detections)

if __name__ == "__main__":
    main()


Adaptive Thresholdings

In [ ]:
import cv2
import matplotlib.pyplot as plt
from inference_sdk import InferenceHTTPClient
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Initialize the Roboflow Client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="IJgd8jCLHIdJE3mNjMvJ"
)

def save_temp_image(image, path):
    """
    Save a thresholded image temporarily for detection.
    Args:
        image (numpy array): Thresholded image.
        path (str): Path to save the image.
    """
    cv2.imwrite(path, image)
    return path

def run_inference(image_path):
    """Run object detection using Roboflow API and return detections."""
    result = CLIENT.infer(image_path, model_id="coco-0qrql/1")
    return result["predictions"]

def display_image_with_boxes(image_path, detections, title):
    """Display a thresholded image with bounding boxes from detections."""
    # Open the image using PIL
    img = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(7, 5))
    ax.imshow(img, cmap="gray")

    # Add bounding boxes
    for detection in detections:
        x, y, w, h = detection["x"], detection["y"], detection["width"], detection["height"]
        label = detection["class"]
        confidence = detection["confidence"]

        # Convert to bounding box (left, top)
        left = x - w / 2
        top = y - h / 2

        # Draw bounding box
        rect = patches.Rectangle((left, top), w, h, linewidth=2, edgecolor="red", facecolor="none")
        ax.add_patch(rect)
        plt.text(left, top - 5, f"{label}: {confidence:.2f}", color="yellow", fontsize=8)

    plt.axis('off')
    plt.title(title)
    plt.show()

def main():
    # Path to the input image
    image_path = r'/content/sample.jpg'

    # Step 1: Load and process the image
    img = cv2.imread(image_path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Step 2: Apply adaptive thresholding
    mean_thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    gaussian_thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Step 3: Save thresholded images temporarily
    mean_thresh_path = save_temp_image(mean_thresh, "mean_thresh.jpg")
    gaussian_thresh_path = save_temp_image(gaussian_thresh, "gaussian_thresh.jpg")

    # Step 4: Run object detection
    print("Running object detection on Mean Adaptive Thresholding...")
    mean_detections = run_inference(mean_thresh_path)
    print("Detections (Mean Thresholding):", mean_detections)

    print("Running object detection on Gaussian Adaptive Thresholding...")
    gaussian_detections = run_inference(gaussian_thresh_path)
    print("Detections (Gaussian Thresholding):", gaussian_detections)

    # Step 5: Display thresholded images with detection results
    display_image_with_boxes(mean_thresh_path, mean_detections, "Mean Adaptive Thresholding Detection")
    display_image_with_boxes(gaussian_thresh_path, gaussian_detections, "Gaussian Adaptive Thresholding Detection")

if __name__ == "__main__":
    main()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from inference_sdk import InferenceHTTPClient
import matplotlib.patches as patches
from PIL import Image

# Initialize the Roboflow Client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="IJgd8jCLHIdJE3mNjMvJ"
)

def region_growing(image, seed_point, threshold):
    """
    Performs region growing on an image.

    Args:
        image: The input image (grayscale).
        seed_point: The coordinates of the seed point.
        threshold: The similarity threshold.

    Returns:
        A binary mask representing the segmented region.
    """
    rows, cols = image.shape
    mask = np.zeros_like(image)
    queue = [seed_point]

    while queue:
        x, y = queue.pop(0)
        if mask[x, y] == 0:
            mask[x, y] = 255  # Mark as segmented
            neighbors = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]
            for nx, ny in neighbors:
                if 0 <= nx < rows and 0 <= ny < cols and abs(int(image[x, y]) - int(image[nx, ny])) <= threshold:
                    queue.append((nx, ny))

    return mask

def save_temp_image(image, path="temp_region_growing.jpg"):
    """
    Saves the region-growing result temporarily.
    """
    cv2.imwrite(path, image)
    return path

def run_inference(image_path):
    """
    Runs inference using Roboflow API.
    """
    result = CLIENT.infer(image_path, model_id="coco-0qrql/1")  # Replace model_id as needed
    return result["predictions"]

def display_detection(image_path, detections):
    """
    Display the region-growing result with bounding boxes.
    """
    img = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(7, 5))
    ax.imshow(img, cmap="gray")

    for detection in detections:
        x, y, w, h = detection["x"], detection["y"], detection["width"], detection["height"]
        label = detection["class"]
        confidence = detection["confidence"]

        # Bounding box
        left = x - w / 2
        top = y - h / 2
        rect = patches.Rectangle((left, top), w, h, linewidth=2, edgecolor="red", facecolor="none")
        ax.add_patch(rect)
        plt.text(left, top - 5, f"{label}: {confidence:.2f}", color="yellow", fontsize=8)

    plt.axis('off')
    plt.title("Region Growing Detection")
    plt.show()

def main():
    # Path to the input image
    image_path = r'/content/sample.jpg'

    # Step 1: Load the image and convert to grayscale
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Apply Region Growing
    seed_point = (100, 100)  # Set a seed point
    threshold = 10  # Similarity threshold
    print("Applying Region Growing...")
    region_mask = region_growing(gray_image, seed_point, threshold)

    # Step 3: Save the segmented mask as a temporary file
    temp_path = save_temp_image(region_mask)

    # Step 4: Run inference on the region-growing result
    print("Running object detection on Region Growing result...")
    detections = run_inference(temp_path)
    print("Detections:", detections)

    # Step 5: Display the Region Growing result with detection bounding boxes
    display_detection(temp_path, detections)

if __name__ == "__main__":
    main()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from inference_sdk import InferenceHTTPClient
import matplotlib.patches as patches
from PIL import Image

# Initialize the Roboflow Client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="IJgd8jCLHIdJE3mNjMvJ"
)

def save_temp_image(image, path="temp_kmeans_segmented.jpg"):
    """
    Save the segmented image temporarily for detection.
    Args:
        image (numpy array): Segmented image.
        path (str): Path to save the image.
    """
    cv2.imwrite(path, image)
    return path

def run_inference(image_path):
    """
    Runs inference using Roboflow API.
    """
    result = CLIENT.infer(image_path, model_id="coco-0qrql/1")  # Replace model_id as needed
    return result["predictions"]

def display_detection(image_path, detections, title="K-Means Segmentation Detection"):
    """
    Display the segmented image with bounding boxes.
    """
    img = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(7, 5))
    ax.imshow(img)

    for detection in detections:
        x, y, w, h = detection["x"], detection["y"], detection["width"], detection["height"]
        label = detection["class"]
        confidence = detection["confidence"]

        # Bounding box
        left = x - w / 2
        top = y - h / 2
        rect = patches.Rectangle((left, top), w, h, linewidth=2, edgecolor="red", facecolor="none")
        ax.add_patch(rect)
        plt.text(left, top - 5, f"{label}: {confidence:.2f}", color="yellow", fontsize=8)

    plt.axis('off')
    plt.title(title)
    plt.show()

def main():
    # Step 1: Load the image
    image_path = r'/content/sample.jpg'
    image = cv2.imread(image_path)

    # Step 2: Reshape the image and apply K-means clustering
    print("Applying K-means clustering...")
    img_reshaped = image.reshape((-1, 3))
    kmeans = KMeans(n_clusters=4, random_state=0)
    kmeans.fit(img_reshaped)

    # Step 3: Get cluster labels and centers
    labels = kmeans.labels_.reshape(image.shape[:2])
    centers = kmeans.cluster_centers_.astype(int)

    # Step 4: Create the segmented image
    segmented_image = np.zeros_like(image)
    for i in range(len(centers)):
        segmented_image[labels == i] = centers[i]

    # Step 5: Save the segmented image temporarily
    segmented_image_path = save_temp_image(segmented_image)

    # Step 6: Run inference on the segmented image
    print("Running object detection on segmented image...")
    detections = run_inference(segmented_image_path)
    print("Detections:", detections)

    # Step 7: Display the segmented image with detection results
    display_detection(segmented_image_path, detections)

if __name__ == "__main__":
    main()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift
from inference_sdk import InferenceHTTPClient
import matplotlib.patches as patches
from PIL import Image

# Initialize the Roboflow Client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="IJgd8jCLHIdJE3mNjMvJ"
)

def save_temp_image(image, path="temp_meanshift_segmented.jpg"):
    """
    Save the segmented image temporarily for detection.
    Args:
        image (numpy array): Segmented image.
        path (str): Path to save the image.
    """
    cv2.imwrite(path, image)
    return path

def run_inference(image_path):
    """
    Runs inference using Roboflow API.
    """
    result = CLIENT.infer(image_path, model_id="coco-0qrql/1")  # Replace model_id as needed
    return result["predictions"]

def display_detection(image_path, detections, title="Mean Shift Segmentation Detection"):
    """
    Display the segmented image with bounding boxes.
    """
    img = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(7, 5))
    ax.imshow(img)

    for detection in detections:
        x, y, w, h = detection["x"], detection["y"], detection["width"], detection["height"]
        label = detection["class"]
        confidence = detection["confidence"]

        # Bounding box
        left = x - w / 2
        top = y - h / 2
        rect = patches.Rectangle((left, top), w, h, linewidth=2, edgecolor="red", facecolor="none")
        ax.add_patch(rect)
        plt.text(left, top - 5, f"{label}: {confidence:.2f}", color="yellow", fontsize=8)

    plt.axis('off')
    plt.title(title)
    plt.show()

def main():
    # Step 1: Load the image and resize
    image_path = r'/content/sample.jpg'
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (100, 100))  # Resize for faster processing

    # Step 2: Reshape the image and apply Mean Shift clustering
    print("Applying Mean Shift clustering...")
    img_reshaped = img_resized.reshape((-1, 3))
    ms = MeanShift()
    ms.fit(img_reshaped)

    # Step 3: Get cluster labels and centers
    labels = ms.labels_.reshape(img_resized.shape[:2])
    centers = ms.cluster_centers_.astype(np.uint8)

    # Step 4: Create the segmented image
    segmented_image = np.zeros_like(img_resized)
    for i in range(len(centers)):
        segmented_image[labels == i] = centers[i]

    # Step 5: Save the segmented image temporarily
    segmented_image_path = save_temp_image(segmented_image)

    # Step 6: Run inference on the segmented image
    print("Running object detection on segmented image...")
    detections = run_inference(segmented_image_path)
    print("Detections:", detections)

    # Step 7: Display the segmented image with detection results
    display_detection(segmented_image_path, detections)

if __name__ == "__main__":
    main()
